In [ ]:
!pip install tensorflow==2.14.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.14.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Assuming your CSV file is named 'file_paths.csv' and located in the root of your Google Drive
file_paths = pd.read_csv('/content/drive/My Drive/dataset3.csv')

In [ ]:
colab_paths = []

for index, row in file_paths.iterrows():
    file_path = row['Image Path']  # Assuming 'file_path' is the column containing the file paths
    print(file_path)
    colab_path = '/content/drive/My Drive/' + file_path  # Assuming the file paths are relative to the root of Google Drive
    colab_paths.append(colab_path)

# Add the new 'colab_path' column to the DataFrame
file_paths['colab_path'] = colab_paths

# Optionally, you can save the DataFrame back to a CSV file with the updated paths
file_paths.to_csv('/content/drive/My Drive/colab_paths1.csv', index=False)

safe/1.png
safe/10.png
safe/100.png
safe/101.png
safe/102.png
safe/103.png
safe/104.png
safe/105.png
safe/106.png
safe/107.png
safe/108.png
safe/109.png
safe/11.png
safe/110.png
safe/111.png
safe/112.png
safe/113.png
safe/114.png
safe/115.png
safe/116.png
safe/117.png
safe/118.png
safe/119.png
safe/12.png
safe/120.png
safe/121.png
safe/122.png
safe/123.png
safe/124.png
safe/125.png
safe/126.png
safe/127.png
safe/128.png
safe/129.png
safe/13.png
safe/130.png
safe/131.png
safe/132.png
safe/133.png
safe/134.png
safe/135.png
safe/136.png
safe/137.png
safe/138.png
safe/139.png
safe/14.png
safe/140.png
safe/141.png
safe/142.png
safe/143.png
safe/144.png
safe/145.png
safe/146.png
safe/147.png
safe/148.png
safe/149.png
safe/15.png
safe/150.png
safe/151.png
safe/152.png
safe/153.png
safe/154.png
safe/155.png
safe/156.png
safe/157.png
safe/158.png
safe/159.png
safe/16.png
safe/160.png
safe/161.png
safe/162.png
safe/163.png
safe/164.png
safe/165.png
safe/166.png
safe/167.png
safe/168.png
safe/169

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/colab_paths1.csv')
train.head()

,Image Path,Class Label,colab_path
0,safe/1.png,0,/content/drive/My Drive/safe/1.png
1,safe/10.png,0,/content/drive/My Drive/safe/10.png
2,safe/100.png,0,/content/drive/My Drive/safe/100.png
3,safe/101.png,0,/content/drive/My Drive/safe/101.png
4,safe/102.png,0,/content/drive/My Drive/safe/102.png


In [ ]:
train_image = []
from tqdm import tqdm
from keras.preprocessing import image
import numpy as np
# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['colab_path'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255.0
    # appending the image to the train_image list
    train_image.append(img)

# converting the list to numpy array
x = np.array(train_image)

# shape of the array
x.shape

100%|██████████| 382/382 [00:05<00:00, 67.79it/s]


(382, 224, 224, 3)

In [ ]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
y = train['Class Label']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

base_model = VGG16(weights='imagenet', include_top=False)

In [ ]:
x_train = base_model.predict(x_train)
x_train.shape

10/10 [==============================] - 207s 20s/step


(305, 7, 7, 512)

In [ ]:
x_test = base_model.predict(x_test)
x_test.shape

3/3 [==============================] - 47s 14s/step


(77, 7, 7, 512)

In [ ]:
x_train = x_train.reshape(305, 7*7*512)
x_test = x_test.reshape(77, 7*7*512)
# x_train_flat = x_train.reshape(x_train.shape[0], -1)
# x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Reshape the flattened arrays to have 8 and 2 samples, respectively, with each sample having a shape of (25088,)
#x_train_flat = x_train_flat[:8]
#x_test_flat = x_test_flat[:2]

print(x_train.shape)
print(x_test.shape)

(305, 25088)
(77, 25088)


In [ ]:
max = x_train.max()
x_train = x_train/max
x_test = x_test/max

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback to save only the weights
mcp_save = ModelCheckpoint('.weights1.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 1024)              25691136  
                                                                 
 dropout_16 (Dropout)        (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 512)               524800    
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 dropout_18 (Dropout)        (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 128)              

In [ ]:
# training the model
vgg_hist = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/50
3/3 [==============================] - 5s 1s/step - loss: 0.7117 - accuracy: 0.4787 - val_loss: 0.6915 - val_accuracy: 0.4805
Epoch 2/50
3/3 [==============================] - 3s 781ms/step - loss: 0.8562 - accuracy: 0.4852 - val_loss: 0.6583 - val_accuracy: 0.5844
Epoch 3/50
3/3 [==============================] - 2s 787ms/step - loss: 0.7842 - accuracy: 0.5213 - val_loss: 0.6447 - val_accuracy: 0.8312
Epoch 4/50
3/3 [==============================] - 2s 772ms/step - loss: 0.8303 - accuracy: 0.5082 - val_loss: 0.6302 - val_accuracy: 0.7013
Epoch 5/50
3/3 [==============================] - 2s 764ms/step - loss: 0.7872 - accuracy: 0.4623 - val_loss: 0.6117 - val_accuracy: 0.7532
Epoch 6/50
3/3 [==============================] - 3s 1s/step - loss: 0.7271 - accuracy: 0.5279 - val_loss: 0.5959 - val_accuracy: 0.9091
Epoch 7/50
3/3 [==============================] - 3s 1s/step - loss: 0.6478 - accuracy: 0.5902 - val_loss: 0.5641 - val_accuracy: 0.7792
Epoch 8/50
3/3 [=============

In [ ]:
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

3/3 [==============================] - 0s 51ms/step - loss: 1.1133 - accuracy: 0.8831
Test Accuracy: 88.31169009208679


In [ ]:
model.save('my_model2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files
files.download('my_model2.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('.weights1.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>